### Importing the packages

In [1]:
import pandas as pd
import os
from collections import Counter, OrderedDict
import operator
import itertools
from itertools import chain

### Importing the two datasets

In [2]:
os.chdir("C:/Users/r.parisi.cons/Desktop/Materiale didattico")
dfalpha = pd.read_csv("tweets.csv",  error_bad_lines=False, sep=";")
dfusers = pd.read_csv("users.csv",  error_bad_lines=False, sep=";")
dfusers[' user id'] = dfusers['user id']
dfusers.drop('user id', inplace=True, axis=1)

### Funzioni:





* delete_retweet(df) : takes as input a dataframe and drop from it all the row where the column 1 (messages' column) starts with "RT" and return a new df 

        
* turn_into_dictionary(df) : takes in input a dataframe and turn it into a dictionary but modifying the messages' column by splittig the text inside (without doing anything else like stemming, tokenization ecc..). Come output da il dizionario creato. 
 
 
* frequencies(docs) : taking as input a dictionary of messagges this function transforme it in another dictionary which has inside all the user (KEY) with the hashtags that they used in their message (considering either tweets and retweets).
 
 
* inverted_index (user_hashtags) : it takes as input the dictionary created in frequencies and create an inverted index (Key: Hashtag; Value : list of Users) 

In [80]:
def delete_retweet (df):
    to_drop = []
    for i in range(len(df)):
        if df.iloc[i][1].startswith("RT"): 
            to_drop.append(i)
    df.drop(to_drop, inplace = True, axis = 0)
    df.reset_index(drop = True, inplace = True)
    return (df)
        
#funzione che trasforma il dataframe in una dictionary con chiave l'id del tweet e come valore ha a sua volta una dictionary che
# prende come chiave l'user id e come valore il messaggio del tweet in questione (splittando le varie parole contenute, in modo 
# da facilitare le operazioni successive, in cui vogliamo estrapolare solo gli hashtags)
def turn_into_dictionary (df):     
    d = df.to_dict("index")
    docs = {}
    for k1 in d.keys():
        docs[k1] = {d[k1][" user id"] : d[k1]["message"].split(" ")}
    return docs    

#in questa funzione prendiamo in input l'output della funzione turn_into_dictionary e restituiamo una dictionary in cui contiamo
#hashtags usati da ogni utente e in più una lista degli hashtag usati in generale

def frequencies (docs):
    hashtags_list = []
    user_hashtags = dict()
    for i in range(len(docs)):
        user = list(docs[i].keys())[0]
        line = docs[i][user]
        hashtags = [word for word in line if word.startswith("#")]
        if len(hashtags)!=0: #alcuni user twittano senza usare gli hashtag, è inutile portarseli dietro
            hashtags_list = hashtags_list+ hashtags
            if user in user_hashtags.keys():
                #user_hashtags[user].update(hashtags)
                user_hashtags[user] = list(set(user_hashtags[user] + hashtags))
            else:
                #user_hashtags[user] = Counter(hashtags)
                user_hashtags[user] = hashtags
    hashtags_count = dict(Counter(hashtags_list))
    return (user_hashtags, hashtags_count)

#inverted index that contains as key the hashtag and as value the list of the user that used it
def inverted_index (user_hashtags, hashtags_count):
    inv_idx = {}
    hashtags = list(hashtag_freq.keys())
    for key in hashtags:
        inv_idx [key] = [user for user in user_hashtags.keys() if key in user_hashtags[user]]
    return inv_idx


### Applying all the function

In [42]:
user_hasht, hashtag_freq = frequencies (turn_into_dictionary(dfalpha))
#i created also hashtag_freq considering that could be usefull to know how many times each users used that hashtag
most_used = sorted(hashtag_freq.items(), key=operator.itemgetter(1), reverse = True)[:3]
inv_idx = inverted_index(user_hasht)
user_to_check = list(itertools.chain.from_iterable([inv_idx[i[0]] for i in most_used]))

In user_msg we have the users' id with the message and we print it

In [82]:
user_msg = [(dfalpha.iloc[i][" user id"],dfalpha.iloc[i]["message"]) for i in range(len(dfalpha)) if dfalpha.iloc[i][" user id"] in user_to_check]
#print(*chain(*user_msg), sep = "\n")

We merge the two dataset since i want to have all the message (excluding the RETWEET) of the users ordered by the number of followers of each users (descendent). We save it in a file without the index

In [74]:
newdf = dfalpha[[" user id", "message"]].merge(dfusers[[" user id", " followers"]], on = " user id")
df_withoutRT = delete_retweet(newdf).sort_values(by= " followers", ascending = False)
#df_withoutRT.reset_index(drop = True, inplace = True)
df_withoutRT.to_csv("FinalAlpha.csv", sep = ";", index = False)